In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qctrlvisualizer qiskit-ibm-catalog

# Q-CTRLのQiskit Functionsを用いた量子位相推定

*使用量の目安: Heron r2プロセッサで約40秒。（注: これは推定値です。実際の実行時間は異なる場合があります。）*
## 背景
量子位相推定（QPE）は、量子コンピューティングにおける基礎的なアルゴリズムであり、Shorのアルゴリズム、量子化学における基底状態エネルギー推定、固有値問題など、多くの重要な応用の基盤となっています。QPEは、ユニタリ演算子の固有状態に関連する位相$\varphi$を推定するもので、以下の関係式に符号化されています。

$$ U \lvert \varphi \rangle = e^{2\pi i \varphi} \lvert \varphi \rangle, $$

$m$個のカウント量子ビットを用いて$\epsilon = O(1/2^m)$の精度で位相を決定します[\[1\]](#references)。これらの量子ビットを重ね合わせ状態に準備し、$U$の制御べき乗を適用した後、逆量子フーリエ変換（QFT）を使用して位相を二進符号化された測定結果として抽出します。QPEは、二進分数が$\varphi$を近似するビット列にピークを持つ確率分布を生成します。理想的な場合、最も確率の高い測定結果は位相の二進展開に直接対応し、他の結果の確率はカウント量子ビット数の増加とともに急速に減少します。しかし、深いQPE回路をハードウェア上で実行するには課題があります。多数の量子ビットとエンタングリング操作により、アルゴリズムはデコヒーレンスやゲートエラーに対して非常に敏感になります。その結果、ビット列の分布が広がったりシフトしたりして、真の固有位相が隠されてしまいます。結果として、最も確率の高いビット列が$\varphi$の正しい二進展開に対応しなくなる可能性があります。

このチュートリアルでは、Q-CTRLのFire Opalエラー抑制およびパフォーマンス管理ツールを使用したQPEアルゴリズムの実装を紹介します。これはQiskit Functionとして提供されています（[Fire Opalドキュメント](/guides/q-ctrl-performance-management)を参照）。Fire Opalは、動的デカップリング、量子ビットレイアウトの改善、エラー抑制技術などの高度な最適化を自動的に適用し、より高忠実度の結果を実現します。これらの改善により、ハードウェアのビット列分布がノイズのないシミュレーションで得られる分布に近づき、ノイズの影響下でも正しい固有位相を確実に特定できるようになります。
## 前提条件
このチュートリアルを始める前に、以下がインストールされていることを確認してください。
- Qiskit SDK v1.4以降（[visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き）
- Qiskit Runtime v0.40以降（`pip install qiskit-ibm-runtime`）
- Qiskit Functions Catalog v0.9.0（`pip install qiskit-ibm-catalog`）
- Fire Opal SDK v9.0.2以降（`pip install fire-opal`）
- Q-CTRL Visualizer v8.0.2以降（`pip install qctrl-visualizer`）
## セットアップ
まず、[IBM Quantum APIキー](http://quantum.cloud.ibm.com/)を使用して認証します。次に、以下のようにQiskit Functionを選択します。（このコードは、ローカル環境に[アカウントが保存済み](/guides/functions#install-qiskit-functions-catalog-client)であることを前提としています。）

In [5]:
from qiskit import QuantumCircuit

import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import qasm2
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qctrlvisualizer as qv
from qiskit_ibm_catalog import QiskitFunctionsCatalog

plt.style.use(qv.get_qctrl_style())

In [ ]:
catalog = QiskitFunctionsCatalog(channel="ibm_quantum_platform")

# Access Function
perf_mgmt = catalog.load("q-ctrl/performance-management")

## ステップ1: 古典的な入力を量子問題にマッピングする
このチュートリアルでは、既知の単一量子ビットユニタリの固有位相を復元するためにQPEを説明します。推定したい位相を持つユニタリは、ターゲット量子ビットに適用される単一量子ビット位相ゲートです。

$$
U(\theta)=
\begin{pmatrix}
1 & 0\\[2pt]
0 & e^{i\theta}
\end{pmatrix}
= e^{i\theta\,|1\rangle\!\langle 1|}.
$$

固有状態$|\psi\rangle=|1\rangle$を準備します。$|1\rangle$は固有値$e^{i\theta}$を持つ$U(\theta)$の固有ベクトルであるため、推定すべき固有位相は以下のようになります。

$$
\varphi = \frac{\theta}{2\pi} \pmod{1}
$$

$\theta=\tfrac{1}{6}\cdot 2\pi$と設定するため、真の位相は$\varphi=1/6$となります。QPE回路は、角度$\theta\cdot2^k$の制御位相回転を適用することで制御べき乗$U^{2^k}$を実装し、その後カウントレジスタに逆QFTを適用して測定します。得られるビット列は$1/6$の二進表現の周辺に集中します。

回路は$m$個のカウント量子ビット（推定精度を設定するため）と1個のターゲット量子ビットを使用します。まず、QPEの実装に必要な構成要素、すなわち量子フーリエ変換（QFT）とその逆変換、固有位相の10進数と二進分数の間の変換を行うユーティリティ関数、シミュレーション結果とハードウェア結果を比較するために生のカウントを確率に正規化するヘルパー関数を定義します。

In [7]:
def inverse_quantum_fourier_transform(quantum_circuit, number_of_qubits):
    """
    Apply an inverse Quantum Fourier Transform the first `number_of_qubits` qubits in the
    `quantum_circuit`.
    """
    for qubit in range(number_of_qubits // 2):
        quantum_circuit.swap(qubit, number_of_qubits - qubit - 1)
    for j in range(number_of_qubits):
        for m in range(j):
            quantum_circuit.cp(-np.pi / float(2 ** (j - m)), m, j)
        quantum_circuit.h(j)
    return quantum_circuit

In [8]:
def bitstring_count_to_probabilities(data, shot_count):
    """
    This function turns an unsorted dictionary of bitstring counts into a sorted dictionary
    of probabilities.
    """
    # Turn the bitstring counts into probabilities.
    probabilities = {
        bitstring: bitstring_count / shot_count
        for bitstring, bitstring_count in data.items()
    }

    sorted_probabilities = dict(
        sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
    )

    return sorted_probabilities

## ステップ2: 量子ハードウェア実行のための問題の最適化
カウント量子ビットを重ね合わせ状態に準備し、制御位相回転を適用してターゲットの固有位相を符号化し、測定前に逆QFTで仕上げることで、QPE回路を構築します。

In [9]:
def quantum_phase_estimation_benchmark_circuit(
    number_of_counting_qubits, phase
):
    """
    Create the circuit for quantum phase estimation.

    Parameters
    ----------
    number_of_counting_qubits : The number of qubits in the circuit.
    phase : The desired phase.

    Returns
    -------
    QuantumCircuit
        The quantum phase estimation circuit for `number_of_counting_qubits` qubits.
    """
    qc = QuantumCircuit(
        number_of_counting_qubits + 1, number_of_counting_qubits
    )
    target = number_of_counting_qubits

    # |1> eigenstate for the single-qubit phase gate
    qc.x(target)

    # Hadamards on counting register
    for q in range(number_of_counting_qubits):
        qc.h(q)

    # ONE controlled phase per counting qubit: cp(phase * 2**k)
    for k in range(number_of_counting_qubits):
        qc.cp(phase * (1 << k), k, target)

    qc.barrier()

    # Inverse QFT on counting register
    inverse_quantum_fourier_transform(qc, number_of_counting_qubits)

    qc.barrier()
    for q in range(number_of_counting_qubits):
        qc.measure(q, q)
    return qc

## ステップ3: Qiskitプリミティブを使用した実行
実験のショット数と量子ビット数を設定し、$m$桁の二進数を使用してターゲット位相$\varphi = 1/6$を符号化します。これらのパラメータを用いて、シミュレーション、デフォルトのハードウェア、およびFire Opal強化バックエンドで実行するQPE回路を構築します。

In [10]:
shot_count = 10000
num_qubits = 35
phase = (1 / 6) * 2 * np.pi
circuits_quantum_phase_estimation = (
    quantum_phase_estimation_benchmark_circuit(
        number_of_counting_qubits=num_qubits, phase=phase
    )
)

### MPSシミュレーションの実行
まず、`matrix_product_state`シミュレータを使用してリファレンス分布を生成し、カウントを正規化された確率に変換して、後でハードウェア結果と比較できるようにします。

In [11]:
# Run the algorithm on the IBM Aer simulator.
aer_simulator = AerSimulator(method="matrix_product_state")

# Transpile the circuits for the simulator.
transpiled_circuits = qiskit.transpile(
    circuits_quantum_phase_estimation, aer_simulator
)

In [12]:
simulated_result = (
    aer_simulator.run(transpiled_circuits, shots=shot_count)
    .result()
    .get_counts()
)

In [13]:
simulated_result_probabilities = []

simulated_result_probabilities.append(
    bitstring_count_to_probabilities(
        simulated_result,
        shot_count=shot_count,
    )
)

### ハードウェアでの実行

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuits = pm.run(circuits_quantum_phase_estimation)

In [15]:
# Run the algorithm with IBM default.
sampler = Sampler(backend)

# Run all circuits using Qiskit Runtime.
ibm_default_job = sampler.run([isa_circuits], shots=shot_count)

### Fire Opalを使用したハードウェアでの実行

In [ ]:
# Run the circuit using the sampler
fire_opal_job = perf_mgmt.run(
    primitive="sampler",
    pubs=[qasm2.dumps(circuits_quantum_phase_estimation)],
    backend_name=backend.name,
    options={"default_shots": shot_count},
)

## ステップ4: 後処理と所望の古典形式での結果の返却

In [ ]:
# Retrieve results.
ibm_default_result = ibm_default_job.result()
ibm_default_probabilities = []

for idx, pub_result in enumerate(ibm_default_result):
    ibm_default_probabilities.append(
        bitstring_count_to_probabilities(
            pub_result.data.c0.get_counts(),
            shot_count=shot_count,
        )
    )

In [ ]:
fire_opal_result = fire_opal_job.result()

fire_opal_probabilities = []
for idx, pub_result in enumerate(fire_opal_result):
    fire_opal_probabilities.append(
        bitstring_count_to_probabilities(
            pub_result.data.c0.get_counts(),
            shot_count=shot_count,
        )
    )

In [16]:
data = {
    "simulation": simulated_result_probabilities,
    "default": ibm_default_probabilities,
    "fire_opal": fire_opal_probabilities,
}

In [21]:
def plot_distributions(
    data,
    number_of_counting_qubits,
    top_k=None,
    by="prob",
    shot_count=None,
):
    def nrm(d):
        s = sum(d.values())
        return {k: (v / s if s else 0.0) for k, v in d.items()}

    def as_float(d):
        return {k: float(v) for k, v in d.items()}

    def to_space(d):
        if by == "prob":
            return nrm(as_float(d))
        else:
            if shot_count and 0.99 <= sum(d.values()) <= 1.01:
                return {
                    k: v * float(shot_count) for k, v in as_float(d).items()
                }
            else:
                return as_float(d)

    def topk(d, k):
        items = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
        return items[: (k or len(d))]

    phase = "1/6"

    sim = to_space(data["simulation"])
    dft = to_space(data["default"])
    qct = to_space(data["fire_opal"])

    correct = max(sim, key=sim.get) if sim else None
    print("Correct result:", correct)

    sim_items = topk(sim, top_k)
    dft_items = topk(dft, top_k)
    qct_items = topk(qct, top_k)

    sim_keys, y_sim = zip(*sim_items) if sim_items else ([], [])
    dft_keys, y_dft = zip(*dft_items) if dft_items else ([], [])
    qct_keys, y_qct = zip(*qct_items) if qct_items else ([], [])

    fig, axes = plt.subplots(3, 1, layout="constrained")
    ylab = "Probabilities"

    def panel(ax, keys, ys, title, color):
        x = np.arange(len(keys))
        bars = ax.bar(x, ys, color=color)
        ax.set_title(title)
        ax.set_ylabel(ylab)
        ax.set_xticks(x)
        ax.set_xticklabels(keys, rotation=90)
        ax.set_xlabel("Bitstrings")
        if correct in keys:
            i = keys.index(correct)
            bars[i].set_edgecolor("black")
            bars[i].set_linewidth(2)
        return max(ys, default=0.0)

    c_sim, c_dft, c_qct = (
        qv.QCTRL_STYLE_COLORS[5],
        qv.QCTRL_STYLE_COLORS[1],
        qv.QCTRL_STYLE_COLORS[0],
    )
    m1 = panel(axes[0], list(sim_keys), list(y_sim), "Simulation", c_sim)
    m2 = panel(axes[1], list(dft_keys), list(y_dft), "Default", c_dft)
    m3 = panel(axes[2], list(qct_keys), list(y_qct), "Q-CTRL", c_qct)

    for ax, m in zip(axes, (m1, m2, m3)):
        ax.set_ylim(0, 1.05 * (m or 1.0))

    for ax in axes:
        ax.label_outer()
    fig.suptitle(
        rf"{number_of_counting_qubits} counting qubits, $2\pi\varphi$={phase}"
    )
    fig.set_size_inches(20, 10)
    plt.show()

In [22]:
experiment_index = 0
phase_index = 0

distributions = {
    "simulation": data["simulation"][phase_index],
    "default": data["default"][phase_index],
    "fire_opal": data["fire_opal"][phase_index],
}

plot_distributions(
    distributions, num_qubits, top_k=100, by="prob", shot_count=shot_count
)

Correct result: 00101010101010101010101010101010101


<Image src="../docs/images/tutorials/quantum-phase-estimation-qctrl/extracted-outputs/593334d2-1.avif" alt="Output of the previous code cell" />

The simulation sets the baseline for the correct eigenphase. Default hardware runs show noise that obscures this result, as noise spreads probability across many incorrect bitstrings. With Q-CTRL Performance Management the distribution becomes sharper and the correct outcome is recovered, enabling reliable QPE at this scale.

## References

[1] Lecture 7: [Phase Estimation and Factoring](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction). IBM Quantum Learning - Fundamentals of quantum algorithms. Retrieved October 3, 2025.

## Tutorial survey

Please take a minute to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3BLFkNVEuh0QBWm)